In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import time
import glob
#import wget
import sys
import os
import re

In [2]:
path, dirs, files = next(os.walk("../data"))
file_count = len(files)
print(file_count)

50


In [3]:
ruta_txt = '../data/*.txt'

In [4]:
def funcion_auxiliar_ponentes(x0,x1,x2,x3):
    if len(x0)!= 0 :
        result = x0
    elif len(x1)!= 0:
        result = x1
    elif len(x2)!= 0:
        result =["No se menciona el nombre del ponente sólo del relator"]
    elif x3 == 1:
        result =["Revisar texto tesis"]    
    return result

In [5]:
def funcion_auxiliar_secretarios(x0,x1,x10):
    if len(x0)!= 0 :
        result = x0
    elif len(x1)!= 0:
        result = ["La publicación no menciona el nombre del ponente"]
    elif x10 == 1:
        result =["Revisar texto tesis"]    
    return result

In [6]:
tabla_tesis_jurisprudencia = pd.DataFrame(columns=['id_tesis', '#_ponente', 'ponente', '#_secretario', 'secretario', 'fecha'])

t0 = time.time()
n = len(glob.glob(ruta_txt))
tipo = []

for i in range(n):

    file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
    text = file.read()
    file.close()
    
    #Eliminar tabulaciones adicionales
    text_aux = text.strip()
    
    #Posicion en el texto de las variables
    pos_registro_digital = text_aux.find('Registro digital:')
    pos_final_registro_digital = text_aux[pos_registro_digital:].find('\n')
    pos_instancia = text_aux.find('Instancia:')
    pos_final_instancia_aux1 = text_aux[pos_instancia:].find('\n\n')

    lista  = []

    lista.append(text_aux[pos_instancia:].find('Primera Época'))
    lista.append(text_aux[pos_instancia:].find('Segunda Época'))
    lista.append(text_aux[pos_instancia:].find('Tercera Época'))
    lista.append(text_aux[pos_instancia:].find('Cuarta Época'))
    lista.append(text_aux[pos_instancia:].find('Quinta Época'))
    lista.append(text_aux[pos_instancia:].find('Sexta Época'))
    lista.append(text_aux[pos_instancia:].find('Séptima Época'))
    lista.append(text_aux[pos_instancia:].find('Octava Época'))
    lista.append(text_aux[pos_instancia:].find('Novena Época'))
    lista.append(text_aux[pos_instancia:].find('Décima Época'))
    lista.append(text_aux[pos_instancia:].find('Undécima Época'))

    pos_final_instancia_aux2 = min([x for x in lista if x > -1])
    
    if pos_final_instancia_aux1 < pos_final_instancia_aux2:
        pos_final_instancia = pos_final_instancia_aux1
    else:
        pos_final_instancia = pos_final_instancia_aux2

    pos_epoca_aux1 = text_aux.find('\n\n')

    lista  = []

    lista.append(text_aux.find('Primera Época'))
    lista.append(text_aux.find('Segunda Época'))
    lista.append(text_aux.find('Tercera Época'))
    lista.append(text_aux.find('Cuarta Época'))
    lista.append(text_aux.find('Quinta Época'))
    lista.append(text_aux.find('Sexta Época'))
    lista.append(text_aux.find('Séptima Época'))
    lista.append(text_aux.find('Octava Época'))
    lista.append(text_aux.find('Novena Época'))
    lista.append(text_aux.find('Décima Época'))
    lista.append(text_aux.find('Undécima Época'))

    pos_epoca_aux2 = min([x for x in lista if x > -1])

    if pos_epoca_aux1 < pos_epoca_aux2:
        pos_epoca = pos_epoca_aux1
    else:
        pos_epoca = pos_epoca_aux2

    if pos_epoca_aux1 < pos_epoca_aux2:
        pos_final_epoca = text_aux[pos_epoca+2:].find('\n\n')
    else:
        pos_final_epoca = text_aux[pos_epoca+2:].find('\n')
        
    pos_materia = text_aux.find('Materia(s):')
    pos_final_materia = text_aux[pos_materia:].find('\n\n')

    pos_tesis = text_aux.find('Tesis:')
    pos_final_tesis = text_aux[pos_tesis:].find('\n\n')

    pos_fuente = text_aux.find('Fuente:')
    pos_final_fuente = text_aux[pos_fuente:].find('.')

    pos_tipo = text_aux.find('Tipo:')
    if text_aux[pos_tipo:].find('\n\n') < text_aux[pos_tipo:].find('\n') :
        pos_final_tipo = text_aux[pos_tipo:].find('\n\n')
    else:
        pos_final_tipo = text_aux[pos_tipo:].find('\n')-2

    pos_encabezado = text_aux[pos_tipo:].find('\n\n')

    #Tamano de las variables
    len_registro_digital = len('Registro digital:')
    len_instancia = len('Instancia:')
    if pos_epoca_aux1 < pos_epoca_aux2:
        len_epoca = len('\n\n')
    else:
        len_epoca = 0
    len_materia = len('Materia(s):')
    len_fuente = len('Fuente:')
    len_tipo = len('Tipo:')
    
    tipo = text_aux[pos_tipo+len_tipo:pos_tipo+pos_final_tipo+2].replace('\n',' ').strip()
    
    if tipo == "Jurisprudencia":
        
        #contradiccion = text_aux.find('Contradicción de tesis')
        #if text_aux.find('Contradicción de tesis') == -1:
        
        if len(re.findall('Ponente',text_aux)) >= 5:
    
            #Búsqueda de Id
            encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
            tesis = [encontrar_id_tesis.group(1)]

            #Limpieza Texto
            text_aux = text_aux.replace('\n\n','###')
            text_aux = text.replace('\n',' ')

            #Búsqueda de párrafo donde se mencione ponente con fecha
            #ponente_fecha_aux = re.findall(r'Ponente: [\s\S]*? \d{2} de [\s\S]*? de \d{4}[\s\S]*?Nota:', text_aux)
            ponente_fecha_aux = re.findall(r'Amparo [\s\S]*? Ponente: [\s\S]*? Secretari\w: [\s\S]*?', text_aux)
            ponente_fecha_texto = ''.join(ponente_fecha_aux)

            #Búsqueda de Ponente
            x0_ponente = re.findall("Ponente: ([\s\S]*?)(?:.###|. Secretari|,|.\nSecretari|;|. Nota:|Pág.|Vol|.Tom|. Véase|. Amparo|. Séptima Época|. Sexta Época)",ponente_fecha_texto) 
            x1_ponente = re.findall("publicación ([\s\S]*?) del ponente",ponente_fecha_texto )
            x2_ponente = re.findall(r"Relator",ponente_fecha_texto)
            x3_ponente = 1
            ponentes = funcion_auxiliar_ponentes(x0_ponente,x1_ponente,x2_ponente,x3_ponente)

            #Búsqueda de Fecha
            fecha_ponente = re.findall(r'\d{1,2} de [\b(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)]*? de \d{4}', ponente_fecha_texto)
            
            #impresión debug
            #print(i)
            #print(tesis)
            #print(fecha_ponente)
            
            #Búsqueda de Secretario
            x0_secretario = re.findall("(?:Secretario:|Secretaria:|Secretario: |Secretaria: |secretario:|secretaria:|secretario: |secretaria: )([\s\S]*?)(\.)",text_aux)
            x0_aux = pd.DataFrame (x0_secretario, columns = ['secretario', 'delete'])
            x0_aux = x0_aux.drop(columns=['delete'])
            x0_secretario = x0_aux['secretario'].values.tolist()
            x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
            x10_secretario = 10
            secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario,x10_secretario)

            #Vector con ID tesis para crear tabla
            vector_tesis = len(ponentes)*tesis
            vector_numero = [(i+1) for i in range(len(ponentes))]

            tabla_iteracion = pd.DataFrame({'id_tesis':vector_tesis,'#_ponente':vector_numero,'ponente':ponentes,'#_secretario':vector_numero, 'secretario':secretarios,'fecha':fecha_ponente})
            tabla_tesis_jurisprudencia = tabla_tesis_jurisprudencia.append(tabla_iteracion)

            # Imprimir cada 10,000 iteraciones
            #if i%1000 == 0:
            #    print("Van {ii} iteraciones de {nn}".format(ii = i, nn = n))

tn = time.time()
print("el tiempo total fue de {tiempo:.2f} segundos".format(tiempo = tn-t0))

el tiempo total fue de 0.08 segundos


In [7]:
tabla_tesis_jurisprudencia

,id_tesis,#_ponente,ponente,#_secretario,secretario,fecha
0,159814,1,Olga Sánchez Cordero de García Villegas,1,Ana Carolina Cienfuegos Posada,26 de octubre de 2005
1,159814,2,Olga Sánchez Cordero de García Villegas,2,Constanza Tort San Román,30 de noviembre de 2005
2,159814,3,Olga Sánchez Cordero de García Villegas,3,Francisco Octavio Escudero Contreras,2 de julio de 2008
3,159814,4,Olga Sánchez Cordero de García Villegas,4,Francisco Octavio Escudero Contreras,3 de diciembre de 2008
4,159814,5,José de Jesús Gudiño Pelayo,5,Carmina Cortés Rodríguez,1 de septiembre de 2010
0,159824,1,Fidelia Camacho Rivera,1,Constantino Baeza León,17 de marzo de 2003
1,159824,2,Leonardo Rodríguez Bastar,2,Isaías Corona Coronado,13 de febrero de 2004
2,159824,3,Gloria García Reyes,3,Fidelia Camacho Rivera,18 de agosto de 2011
3,159824,4,José Luis Caballero Rodríguez,4,Zazil Ha Hernández Contreras,2 de septiembre de 2011
4,159824,5,José Luis Caballero Rodríguez,5,Zazil Ha Hernández Contreras,2 de septiembre de 2011


La tabla debe llevar esta info: Número de registro de tesis, instancia, época, materia, fecha, ponente, secretario.

#### Pruebas para tesis 159814 y 159824 (usado para crear el código anterior)

In [8]:
file_ej1 = open(glob.glob(ruta_txt)[7],'r', encoding='utf8')
text_ej1 = file_ej1.read()
file_ej1.close()

file_ej2 = open(glob.glob(ruta_txt)[17],'r', encoding='utf8')
text_ej2 = file_ej2.read()
file_ej2.close()

In [9]:
text_aux_ej1 = text_ej1.strip()
text_aux_ej2 = text_ej2.strip()

In [10]:
text_aux_ej1 = text_aux_ej1.replace('\n\n','###')
text_aux_ej1 = text_aux_ej1.replace('\n',' ')
#text_aux_ej1

In [11]:
text_aux_ej2 = text_aux_ej2.replace('\n\n','###')
text_aux_ej2 = text_aux_ej2.replace('\n',' ')
#text_aux_ej2

In [12]:
text_aux2_ej1 = text_ej1.strip()
text_aux2_ej1 = text_aux2_ej1.replace('Pág. 1 de 1','')
text_aux2_ej1 = text_aux2_ej1.replace('Pág. 1 de 2','')
text_aux2_ej1 = text_aux2_ej1.replace('Pág. 2 de 2','')
text_aux2_ej1 = text_aux2_ej1.replace('Semanario Judicial de la Federación','')
text_aux2_ej1 = text_aux2_ej1.replace('http://sjf2.scjn.gob.mx/detalle/tesis/','')
text_aux2_ej1 = text_aux2_ej1.replace('Fecha de impresión','')
#text_aux2_ej1 = re.sub('\d', '',text_aux2_ej1)

text_aux2_ej1 = text_aux2_ej1.replace('\n\n','###')
text_aux2_ej1 = text_aux2_ej1.replace('\n',' ')
text_aux2_ej1 = text_aux2_ej1.replace('### //###\x0c','')

text_aux2_ej1

' Tesis Registro digital: 159814 Instancia: Primera Sala###Décima Época###Materia(s): Constitucional, Penal###Tesis: 1a./J. 71/2014 (9a.)###Fuente: Gaceta del Semanario Judicial de la Federación. Libro 12, Noviembre de 2014, Tomo I, página 585###Tipo: Jurisprudencia###OPERACIONES CON RECURSOS DE PROCEDENCIA ILÍCITA. EL ARTÍCULO 400 BIS, PÁRRAFO SEXTO, DEL CÓDIGO PENAL FEDERAL, VIGENTE HASTA EL 14 DE MARZO DE 2014, NO VULNERA EL PRINCIPIO DE PRESUNCIÓN DE INOCENCIA.###El precepto citado, al prever que son producto de una actividad ilícita los recursos, derechos o bienes de cualquier naturaleza, cuando existan indicios fundados o certeza de que provienen directa o indirectamente, o representan las ganancias derivadas de la comisión de algún delito y no pueda acreditarse su legítima procedencia, no vulnera el principio de presunción de inocencia, pues la determinación de tal elemento normativo exige comprobar que la actividad de la que proceden los recursos es ilícita, para lo cual, el Mi

**Ponentes y Fechas**

In [13]:
ponente_fecha_aux_ej1 = re.findall(r'Amparo [\s\S]*? Ponente: [\s\S]*? Secretari\w: [\s\S]*?', text_aux_ej1)
ponente_fecha_texto_ej1 = ''.join(ponente_fecha_aux_ej1)
ponente_fecha_texto_ej1
ponente_fecha_aux_ej1

['Amparo en revisión 1374/2005. 26 de octubre de 2005. Cinco votos de los Ministros José de Jesús Gudiño Pelayo, Sergio A. Valls Hernández, Juan N. Silva Meza, José Ramón Cossío Díaz, quien formuló voto concurrente y Olga Sánchez Cordero de García Villegas. Ponente: Olga Sánchez Cordero de García Villegas. Secretaria: ',
 'Amparo directo en revisión 1760/2005. 30 de noviembre de 2005. Cinco votos de los Ministros José de Jesús Gudiño Pelayo, Sergio A. Valls Hernández, Juan N. Silva Meza, José Ramón Cossío Díaz y Olga Sánchez Cordero de García Villegas. Ponente: Olga Sánchez Cordero de García Villegas. Secretaria: ',
 'Amparo directo en revisión 794/2008. 2 de julio de 2008. Cinco votos de los Ministros José de Jesús Gudiño Pelayo, José Ramón Cossío Díaz, Juan N. Silva Meza, Olga Sánchez Cordero de García Villegas y Sergio A. Valls Hernández. Ponente: Olga Sánchez Cordero de García Villegas. Secretario: ',
 'Amparo en revisión 327/2008. 3 de diciembre de 2008. Mayoría de cuatro votos de

In [14]:
#Búsqueda de Ponente
x_0_ej1 = re.findall("Ponente: ([\s\S]*?)(?:.###|. Secretari|,|.\nSecretari|;|. Nota:|Pág.|Vol|.Tom|. Véase|. Amparo|. Séptima Época|. Sexta Época)",
                 ponente_fecha_texto_ej1) 
x_1_ej1 = re.findall("publicación ([\s\S]*?) del ponente",ponente_fecha_texto_ej1)
x_2_ej1 = re.findall(r"Relator",ponente_fecha_texto_ej1)
x_3_ej1 = 1

ponentes_ej1 = funcion_auxiliar_ponentes(x_0_ej1,x_1_ej1,x_2_ej1,x_3_ej1)
ponentes_ej1

['Olga Sánchez Cordero de García Villegas',
 'Olga Sánchez Cordero de García Villegas',
 'Olga Sánchez Cordero de García Villegas',
 'Olga Sánchez Cordero de García Villegas',
 'José de Jesús Gudiño Pelayo']

In [15]:
#Búsqueda de Fecha
fecha_ponente_ej1 = re.findall(r'\d{1,2} de [\b(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)]*? de \d{4}',
                               ponente_fecha_texto_ej1)
fecha_ponente_ej1

['26 de octubre de 2005',
 '30 de noviembre de 2005',
 '2 de julio de 2008',
 '3 de diciembre de 2008',
 '1 de septiembre de 2010']

In [16]:
ponente_fecha_aux_ej2 = re.findall(r'Amparo [\s\S]*? Ponente: [\s\S]*? Secretari\w: [\s\S]*?', text_aux_ej2)
ponente_fecha_texto_ej2 = ''.join(ponente_fecha_aux_ej2)
ponente_fecha_texto_ej2

'Amparo directo 102/2003. Sección 26 del Sindicato de Trabajadores Petroleros de la República Mexicana y otro. 17 de marzo de 2003. Unanimidad de votos. Ponente: Fidelia Camacho Rivera, Pág. 1 de 2 http://sjf2.scjn.gob.mx/detalle/tesis/159824###Fecha de impresión 09/07/2021###\x0cSemanario Judicial de la Federación secretaria de tribunal autorizada por el Pleno del Consejo de la Judicatura Federal para desempeñar las funciones de Magistrada. Secretario: Amparo directo 1066/2003. Blanca Aurora Calderón Velazco. 13 de febrero de 2004. Unanimidad de votos. Ponente: Leonardo Rodríguez Bastar. Secretario: Amparo directo 595/2011. Juan Carlos Jiménez de la Cruz. 18 de agosto de 2011. Unanimidad de votos. Ponente: Gloria García Reyes. Secretaria: Amparo directo 621/2011. Carmita Jiménez Jiménez. 2 de septiembre de 2011. Unanimidad de votos. Ponente: José Luis Caballero Rodríguez. Secretaria: Amparo directo 622/2011. 2 de septiembre de 2011. Unanimidad de votos. Ponente: José Luis Caballero Ro

In [17]:
#Búsqueda de Ponente
x_0_ej2 = re.findall("Ponente: ([\s\S]*?)(?:.###|. Secretari|,|.\nSecretari|;|. Nota:|Pág.|Vol|.Tom|. Véase|. Amparo|. Séptima Época|. Sexta Época)",
                 ponente_fecha_texto_ej2) 
x_1_ej2 = re.findall("publicación ([\s\S]*?) del ponente",ponente_fecha_texto_ej2)
x_2_ej2 = re.findall(r"Relator",ponente_fecha_texto_ej2)
x_3_ej2 = 1

ponentes_ej2 = funcion_auxiliar_ponentes(x_0_ej2,x_1_ej2,x_2_ej2,x_3_ej2)
ponentes_ej2

['Fidelia Camacho Rivera',
 'Leonardo Rodríguez Bastar',
 'Gloria García Reyes',
 'José Luis Caballero Rodríguez',
 'José Luis Caballero Rodríguez']

In [18]:
#Búsqueda de Fecha
fecha_ponente_ej2 = re.findall(r'\d{1,2} de [\b(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)]*? de \d{4}',
                               ponente_fecha_texto_ej2)
fecha_ponente_ej2

['17 de marzo de 2003',
 '13 de febrero de 2004',
 '18 de agosto de 2011',
 '2 de septiembre de 2011',
 '2 de septiembre de 2011']

**Secretarios**

In [19]:
def funcion_auxiliar_secretarios(x_0,x_1,x_10):
    if len(x_0)!= 0 :
        result = x_0
    elif len(x_1)!= 0:
        result = ["La publicación no menciona el nombre del ponente"]
    elif x_10 == 1:
        result =["Revisar texto tesis"]    
    return result

In [20]:
prueba = re.findall("(?:Secretario:|Secretaria:|Secretario: |Secretaria: |secretario:|secretaria:|secretario: |secretaria: )([\s\S]*?)(\.)",text_aux2_ej1)
prueba

[(' Ana Carolina Cienfuegos Posada', '.'),
 (' Constanza Tort San Román', '.'),
 (' Francisco Octavio Escudero Contreras', '.'),
 (' Francisco Octavio Escudero Contreras', '.'),
 (' Carmina Cortés Rodríguez', '.')]

In [21]:
#Original del equipo anterior
x_0 = re.findall("(?:Secretario:|Secretaria:|Secretario: |Secretaria: |secretario:|secretaria:|secretario: |secretaria: )([\s\S]*?)(?:\.###|se encargo|Quinta Epoca:|. Hay cosa|Acumulación|Impedimento|Antecedente:|Revisión|Expediente|Los señores|Jurisprudencia|demás que las|XIV.|las demás que|fallado|no superan|Aplicada|De conformidad|contra el|para el|Lo resolvió|Secretarrio:|Presidente|Secretario encargado|ha estimado|Disidente.|Veáse|Impedido:|Vease:|Por |Sexta Epoca|Integró|Para|Tribunal|Ausente|votos|_________________|Texto|Secretarios|Reproduce|Véanse:|Ponente:|EL|Tomo|Resuelto|Ausentes|Que|Varios|Con|Aclaración|Tomando|Juicio|Relator|Ministro|Incidente|/.|Texto aprobado|Relacionada|Séptima|Acción|Facultad|Controversia|Solicitud|Sostiene|La |Inejecución|Octava|En |Este |Se |Repetición|Reitera|El |Contradicción|La presente|El Tribunal|Tesis|La Primera|Sostienen|Informe|Véase|Esta|Importa|Disidentes:|Voto|Ausente:|Secretario:|###|Inconformidad|Conflicto|Nota|Criterios|Reconocimiento|Competencia|Precedente|Recurso|Secretaria|Amparo|;|\. Nota\:|\. Notas\:|Nota\:|Pág\.|Vol|\. Volumen|\. Volúme|\.Tom|\. Véase|\. Amparo di|\.   Amparo dir|\. Amparo in|Amparo en|\. Undécima Época|\. Décima Época|\. Novena Época|\. Octava Época|\. Séptima Época|\. Sexta Época|\. Quinta Época|\. Cuarta Época|\. Tercera Época|\. Segunda Época|\. Primera Época|\. Undécima Epoca|\. Décima Epoca|\. Novena Epoca|\. Octava Epoca|\. Séptima Epoca|\. Sexta Epoca|\. Quinta Epoca|\. Cuarta Epoca|\. Tercera Epoca|\. Segunda Epoca|\. Primera Epoca|\. Improcedencia civil|Sostiene la misma tesis:|\. Revisión fiscal|\. Unanimidad|\. Queja|Queja|\. Competencia|\. Reclamación|\.   Reclamación /.|\.  de enero de|\.  de febrero de|\.  de marzo de|\.  de abril de|\.  de mayo de|\.  de junio de|\.  de julio de|\.  de agosto de|\.  de septiembre de|\.  de octubre de|\.  de noviembre de|\.  de diciembre de|\. Disidente:|\. Precedentes|\. Improcedencia|\. Impedimento|\. Excusa|\. Incidente|\. Encargad|\. Revisión|\. Tomo|por licencia concedida|en funciones de Magistrado por Ministerio de Ley|\. Engrose: |\. Criterios|\. Tesis|\.   Tesis|\. El Tribunal|\. Contradicción de tesis|\. Texto|\. NOTA|\. Recurso de|\.   El Tribunal|\. Véanse|\.   Revisión|.   Notas)",text_aux2_ej1)                                
x_0

[' Ana Carolina Cienfuegos Posada',
 ' ',
 ' Francisco Octavio Escudero ',
 ' Francisco Octavio Escudero ',
 ' Carmina Cortés Rodríguez']

In [22]:
#Modificado
x_0 = re.findall("(?:Secretario:|Secretaria:|Secretario: |Secretaria: |secretario:|secretaria:|secretario: |secretaria: )([\s\S]*?)(\.)",text_aux2_ej1)
x0_aux = pd.DataFrame (x_0, columns = ['secretario', 'delete'])
x0_aux = x0_aux.drop(columns=['delete'])
x_0 = x0_aux['secretario'].values.tolist()
x_0

[' Ana Carolina Cienfuegos Posada',
 ' Constanza Tort San Román',
 ' Francisco Octavio Escudero Contreras',
 ' Francisco Octavio Escudero Contreras',
 ' Carmina Cortés Rodríguez']

In [23]:
#Modificado no usado
#x_0 = re.findall("(?:Secretario:|Secretaria:|Secretario: |Secretaria: |secretario:|secretaria:|secretario: |secretaria: )([\s\S]*?)(\.)",text_aux2_ej1)
#x0_aux = np.array(x_0)
#x0_aux = np.delete(x0_aux, 1, 1)
#x_0 = x0_aux.tolist()
#x_0

In [24]:
x_1 = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux2_ej1)
x_1

[]

In [25]:
x_10 = 1

In [26]:
secretarios = funcion_auxiliar_secretarios(x_0,x_1,x_10)
secretarios

[' Ana Carolina Cienfuegos Posada',
 ' Constanza Tort San Román',
 ' Francisco Octavio Escudero Contreras',
 ' Francisco Octavio Escudero Contreras',
 ' Carmina Cortés Rodríguez']

In [27]:
vector_tesis = len(secretarios)*tesis
vector_numero_secretario = [(i+1) for i in range(len(secretarios))]

In [28]:
tabla_iteracion = pd.DataFrame({'id_tesis':vector_tesis,'secretario':secretarios,'numero_secretario':vector_numero_secretario})

In [29]:
tabla_iteracion

,id_tesis,secretario,numero_secretario
0,159824,Ana Carolina Cienfuegos Posada,1
1,159824,Constanza Tort San Román,2
2,159824,Francisco Octavio Escudero Contreras,3
3,159824,Francisco Octavio Escudero Contreras,4
4,159824,Carmina Cortés Rodríguez,5
